In [20]:
"""
Author: Ngo Van Uc
Date:
Contact: ngovanuc.1508@gmail.com
"""

'\nAuthor: Ngo Van Uc\nDate: \nContact: ngovanuc.1508@gmail.com\n'

In [21]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [22]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# 01. Preprocessing

In [23]:
train_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Train.csv'
test_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Test.csv'
dev_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [24]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9}
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [25]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [26]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]] * 10

train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [27]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 02. Build data loader

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [29]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [30]:
dataset.__len__()

7786

# 03. Build Transformers model

In [31]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class, n_heads=8, n_layers=6):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')

        # transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=n_heads, dim_feedforward=2048)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.classifier = nn.Linear(786, n_classes*n_labels_per_class) # 786 là d_model của BERT

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Apply Transformers encoder to BERT's last hidden states
        transformer_outputs = self.transformer(bert_outputs.last_hidden_state)

        # Lấy output của bước cuối cùng
        transformer_outputs = transformer_outputs[:, -1, :] # last token output
        output = self.classifier(transformer_outputs)

        return output.view(-1, 10, 4)

model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

In [32]:
pip install torch==1.13.1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model.train()  # Bỏ gán lại model = model.train(), chỉ cần gọi model.train()
    losses = []

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


# Cấu hình thiết bị và mô hình
EPOCHS = 40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True

# Định nghĩa Optimizer và Loss Function
optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

# Giả sử `dataloader` được định nghĩa trước đó
if training:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')

        if epoch % 5 == 0:
            torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_Transformers_model_{epoch}.pth')


In [33]:
import torch.optim as optim


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')
        if epoch % 5 == 0:
            torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_Transformers_model_{epoch}.pth')

ModuleNotFoundError: No module named 'torch._dynamo'

In [ ]:
path_save_model = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models'
torch.save(model.state_dict(), path_save_model+'/ABSA_using_Transformers_model.pth')

In [ ]:
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/ABSA_using_Transformers_model.pth'))
# model.eval()

# 04. Predict

In [ ]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên toàn bộ dữ liệu kiểm thử (testset)

In [ ]:
test_data = pd.read_csv(test_path)
test_data.head()

In [ ]:
max_len = 256
idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

In [ ]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


In [ ]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [ ]:
def full_label(true_label, prediction):
    '''true_label: dict
     prediction: dict '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [ ]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = []
for idx in range(len(predictions_on_test)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [ ]:
# data thỉnh thoảng bị sai -..-
# bằng chứng là SER&ACC trong câu này phải là Positive nhưng nhãn là None
# hoặc GENERAL nên là Positive thì nhãn lại là None

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

# 07. Đánh giá hiệu suất tổng thể của mô hình

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx][key] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and true_full_labels[idx][key] == value:
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy, Precision, Recall, F1-score trên toàn bộ khía cạnh

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

print(f'Accuracy = {accuracy_score(true_, predicted_)}')
print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

### đánh giá hiệu suất khía cạnh trên class SCREEN

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SCREEN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class FEATURES

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'FEATURES'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PERFORMANCE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class GENERAL

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'GENERAL'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class BATTERY

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'BATTERY'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class STORAGE

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'STORAGE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class DESIGN

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'DESIGN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class PRICE

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PRICE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SER&ACC'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

### đánh giá hiệu suất khía cạnh trên class CAMERA

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'CAMERA'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [ ]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_full_labels)):
    for key, value in true_full_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)

print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

### Precisio, Recall, F1-score tình cảm trên toàn bộ data kiểm thử

In [ ]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

In [ ]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [ ]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [ ]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [ ]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [ ]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [ ]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [ ]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [ ]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [ ]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [ ]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [ ]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)